In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable

In [2]:
#导入数据
factor_df=pd.read_csv("/data01/data_for_intern/factor_data_13_19.csv")
return_df=pd.read_csv("/data01/data_for_intern/return_data_13_19.csv")
return_label=return_df.drop(columns=['return_adj_d001','return_adj_d005','car_hs300_b30_d005'])
return_label['car_hs300_b30_d001']=(return_label['car_hs300_b30_d001']>=0)*1
#利用内连接方式合并数据
merged_factor_return = factor_df.merge(return_label,on=["stock_code","trade_date"],how="inner")
merged_factor_return

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
0,000001.SZ,2013-01-04,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,...,0.790698,0.724551,0.812212,20.869800,43.5275,0.741100,0.680235,1.57469,3.43924,1
1,000001.SZ,2013-01-07,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,...,0.856688,0.909091,0.841909,18.507200,41.6409,0.741100,0.704779,1.47261,3.15030,0
2,000001.SZ,2013-01-08,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,...,0.676724,0.727273,0.786972,20.082300,29.5337,0.605634,0.705782,1.48008,3.02077,0
3,000001.SZ,2013-01-09,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,...,0.632231,0.642424,0.759596,8.662940,27.9438,0.536232,0.699468,1.47922,3.01475,0
4,000001.SZ,2013-01-10,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,...,0.629167,0.516667,0.628788,12.206900,27.2890,0.520000,0.690864,1.51058,3.06918,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088415,603999.SH,2019-12-13,0.063063,0.007508,0.070571,-9.0,-0.875646,-0.32,0.138746,-0.876772,...,0.542857,0.855769,0.822900,1.148900,20.9302,0.029940,0.307711,1.14262,3.09775,1
5088416,603999.SH,2019-12-16,0.038682,0.007163,0.045845,-9.0,-0.880784,-0.18,0.306574,-0.751634,...,0.583333,0.884298,0.811730,0.782227,27.0718,0.187879,0.276938,1.04110,2.77682,0
5088417,603999.SH,2019-12-17,0.002821,0.016925,0.019746,-8.0,-0.854110,0.11,0.863933,-0.479172,...,0.654971,0.778761,0.839609,0.342224,29.0801,0.298013,0.260858,1.04584,2.66659,0
5088418,603999.SH,2019-12-18,0.010029,0.012894,0.022923,-7.0,-0.733061,0.04,0.511944,-0.716503,...,0.670659,0.736364,0.799807,0.391114,29.2537,0.324324,0.252109,1.07986,2.67018,1


In [8]:
#举例
temp=pd.DataFrame(merged_factor_return.loc[0:9]).drop(['stock_code','trade_date','car_hs300_b30_d001'],axis=1)
temp=np.array(temp.values)
train_data=temp.reshape(1,10,42)
train_data.shape

(1, 10, 42)

In [9]:
#dataframe预处理
df=pd.DataFrame(merged_factor_return).drop(['trade_date','car_hs300_b30_d001'],axis=1)
#筛选股票名称
name=np.unique(merged_factor_return['stock_code'].values)

In [11]:
#整理训练数据
waste=[]
for idx in name:
    temp_df=df[df['stock_code']==idx]
    bound=temp_df.shape[0]-300 #留300-10天记录作为valid_data和test_data
    if bound<0:
        waste.append(idx)
        continue
    temp_df=temp_df.drop(['stock_code'],axis=1)
    for i in range(0,bound):
        temp=temp_df.iloc[i:i+10]
        temp=np.array(temp.values).reshape(1,10,42)
        train_data=np.concatenate((train_data,temp),axis=0)

KeyboardInterrupt: 

In [21]:
idx,train_data.shape#'000820.SZ'

('000416.SZ', (468213, 10, 42))

In [22]:
np.save('new_train_data',train_data)

In [23]:
clean_name = np.setdiff1d(name,waste)
clean_name

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '603997.SH',
       '603998.SH', '603999.SH'], dtype=object)

In [24]:
#整理valid数据
temp=pd.DataFrame(merged_factor_return.loc[1694-300:1403]).drop(['stock_code','trade_date','car_hs300_b30_d001'],axis=1)
temp=np.array(temp.values)
valid_data=temp.reshape(1,10,42)
valid_data.shape

(1, 10, 42)

In [25]:
for idx in clean_name:
    if idx=='000820.SZ':
        break
    temp_df=df[df['stock_code']==idx]
    bound=temp_df.shape[0]-300 #留300-10天记录作为valid_data和test_data
    temp_df=temp_df.drop(['stock_code'],axis=1)
    for i in range(bound,bound+150):
        temp=temp_df.iloc[i:i+10]
        temp=np.array(temp.values).reshape(1,10,42)
        valid_data=np.concatenate((valid_data,temp),axis=0)

In [26]:
#整理train_label
train_label=np.zeros(1,)
train_label[0]=merged_factor_return['car_hs300_b30_d001'].loc[10]
waste_sec=[]
for idx in name:
    if idx=='000820.SZ':
        break
    temp_df=merged_factor_return[merged_factor_return['stock_code']==idx]
    bound=temp_df.shape[0]-300 
    if bound<0:
        waste_sec.append(idx)
        continue
    for i in range(10,bound+10):
        temp=np.array(temp_df['car_hs300_b30_d001'].iloc[i])
        train_label=np.concatenate((train_label,temp.reshape(1,)),axis=0)

In [33]:
#整理valid_label
valid_label=np.zeros(1,)
valid_label[0]=merged_factor_return['car_hs300_b30_d001'].loc[1404]
for idx in clean_name:
    if idx=='000820.SZ':
        break
    temp_df=merged_factor_return[merged_factor_return['stock_code']==idx]
    bound=temp_df.shape[0]-320 
    for i in range(bound+10,bound+160):
        temp=np.array(temp_df['car_hs300_b30_d001'].iloc[i])
        valid_label=np.concatenate((valid_label,temp.reshape(1,)),axis=0)

In [30]:
np.save('new_valid_data',valid_data)
np.save('new_train_label',train_label)
np.save('new_valid_label',valid_label)

In [34]:
valid_label.shape

(51001,)

In [32]:
valid_data.shape

(51001, 10, 42)

In [35]:
train_label.shape

(467644,)

In [39]:
np.save('new_train_data',train_data[0:train_label.shape[0],:,:])